In [1]:
import requests
from bs4 import BeautifulSoup as bs
import json
import re

In [2]:
NUM_PAGES = 31
data = []

In [3]:
def scrapePage(page):
    page_data = []

    facilities_source = requests.get('https://facilities.umd.edu/info-resources/building-inventory?page=' + str(page))
    soup = bs(facilities_source.content)
    for bldg in soup.find_all('article'):
        bldg_data = {}

        bldg_id = bldg.find(class_='bldg-top').find('a').contents[0].string
        match = re.match(r"(\d{3}) - (.*)", bldg_id)

        bldg_data['id'] = [match[1]]
        bldg_data['name'] = [match[2].title()]

        bldg_info = bldg.find(id='bldg-info')
        for field in bldg_info.find_all(class_='field'):
            label = field.find(class_='field__label').contents[0].string
            value = field.find(class_='field__item')
            content = value.attrs.get('content')
            value = value.contents[0]
            if value.string == None:
                value = value.contents[0].string
            else:
                value = value.string
            if content:
                bldg_data[label] = [value, float(content)]
            else:
                bldg_data[label] = [value]
        page_data.append(bldg_data)
    data.extend(page_data)

In [4]:
for page in range(NUM_PAGES):
    scrapePage(page)

with open('allFacilities.json', 'w') as f:
    json.dump(data, f)